# 多智能体 NLP 项目：架构与实现

本项目构建了一个集成了大脑、规划、工具和记忆等组件的复杂双智能体学术写作优化系统，包含知识蒸馏与数据合成功能。

**核心组件:**
1.  **环境设置**: 安装必要的 Python 库。
2.  **API 密钥配置**: 安全地加载 API 密钥。
3.  **智能体架构**:
    *   **大脑**: 集成 GPT-4 LLM。
    *   **工具**: 定义网络搜索、代码执行和文件操作等工具。
    *   **记忆**: 使用 FAISS 向量数据库实现长期记忆。
    *   **双智能体系统**: Agent A (优化者) 和 Agent B (批评者) 协作。
4.  **知识蒸馏**: 专家指导与Agent学习系统。
5.  **持久化知识**: 知识存储与动态应用机制。

## 1. 环境设置与依赖项安装

此部分代码将使用 `pip` 安装所有必需的 Python 库。为了运行此项目，你需要 `openai` (与 GPT-4 交互), `langchain` (构建智能体的主要框架), `google-search-results` (用于网络搜索), `faiss-cpu` (向量数据库), `tiktoken` (用于文本分词) 等。

In [1]:
%pip install openai langchain langchain-openai langchain-community langchain-experimental langchain-core google-search-results faiss-cpu tiktoken python-dotenv


Note: you may need to restart the kernel to use updated packages.


## 2. API 密钥与配置

**极其重要**: 为了保护你的 API 密钥，我们强烈建议使用环境变量来加载它们，而不是将它们硬编码在代码中。

**操作步骤**:
1.  在项目根目录下创建一个名为 `.env` 的文件。
2.  将你的 API 密钥添加到 `.env` 文件中，格式如下：
    ```
    OPENAI_API_KEY="YOUR_OPENAI_API_KEY"
    SERPAPI_API_KEY="YOUR_SERPAPI_API_KEY"
    ```
3.  **确保将 `.env` 文件添加到你的 `.gitignore` 文件中**，以防止它被提交到版本控制系统。

下面的代码将使用 `python-dotenv` 库来加载这些密钥。在代码中，我们使用 `"API_KEY"` 作为占位符，你**必须**在运行前替换它们。

In [2]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()

# 获取 API 密钥
openai_api_key = os.getenv("OPENAI_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

# 详细的配置检查
print("🔍 API 密钥配置检查:")
print("-" * 30)

if not openai_api_key:
    print("❌ OpenAI API 密钥 (OPENAI_API_KEY) 未设置")
    print("   请在 .env 文件中添加: OPENAI_API_KEY=\"您的密钥\"")
elif openai_api_key.startswith("sk-") and len(openai_api_key) > 20:
    print("✅ OpenAI API 密钥格式正确")
else:
    print("⚠️  OpenAI API 密钥格式可能不正确")
    print("   密钥应该以 'sk-' 开头且长度较长")

if not serpapi_api_key:
    print("❌ SerpAPI 密钥 (SERPAPI_API_KEY) 未设置")
    print("   请在 .env 文件中添加: SERPAPI_API_KEY=\"您的密钥\"")
elif len(serpapi_api_key) > 10:
    print("✅ SerpAPI 密钥已设置")
else:
    print("⚠️  SerpAPI 密钥长度可能不正确")

print("-" * 30)

if openai_api_key and serpapi_api_key:
    if (openai_api_key.startswith("sk-") and 
        openai_api_key != "YOUR_OPENAI_API_KEY" and 
        serpapi_api_key != "YOUR_SERPAPI_API_KEY"):
        print("🎉 API 密钥配置完成！准备开始...")
    else:
        print("⚠️  请确保使用真实的API密钥，不是占位符")
else:
    print("❌ 请先配置 API 密钥才能继续")

🔍 API 密钥配置检查:
------------------------------
✅ OpenAI API 密钥格式正确
✅ SerpAPI 密钥已设置
------------------------------
🎉 API 密钥配置完成！准备开始...


## 3. 大脑：集成 LLM (GPT-4o-mini)

智能体的"大脑"是其核心决策组件。在这里，我们使用 LangChain 框架来实例化一个强大的大型语言模型（LLM），使用 GPT-4o-mini。这个模型性能优秀，免费版本提供 200次/天 的调用额度，非常适合智能体开发。它将负责理解用户输入、进行推理和生成响应。

In [3]:
from langchain_openai import ChatOpenAI

# 初始化 GPT-4o-mini LLM (大脑)
# 使用 ChatAnywhere 提供的转发服务，需要设置 base_url  
# gpt-4o-mini: 200次/天，性能优秀，适合智能体开发
llm = ChatOpenAI(
    temperature=0, 
    model_name="gpt-4o-mini",
    openai_api_key=openai_api_key,
    base_url="https://api.chatanywhere.tech/v1"  # 国内中转，延时更低
)

print("LLM '大脑' 已配置完成。")

LLM '大脑' 已配置完成。


## 4. 工具：定义与集成

为了让智能体能够与外部世界互动并执行具体任务，我们需要为它提供一套“工具”。这些工具可以是任何函数或 API 调用。

-   **网络搜索**: 使用 `SerpAPI` 来获取实时信息。
-   **代码执行**: 使用 `Python REPL` 来执行动态生成的 Python 代码。
-   **文件读写**: (为简化，此处我们用一个自定义工具来模拟)

In [4]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.utilities import SerpAPIWrapper

# 1. 网络搜索工具 (使用 SerpAPI)
search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)
search_tool = Tool(
    name="网络搜索",
    func=search.run,
    description="当你需要回答关于时事、天气或任何需要实时信息的查询时,应使用此工具。输入应该是一个搜索查询。"
)

# 2. Python 代码执行工具
python_repl = PythonREPL()
python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="一个 Python shell。当你需要执行 Python 代码来回答问题时,应使用此工具。输入应该是格式正确的 Python 代码。"
)

# 3. 自定义文件读写工具
def read_file(filename: str) -> str:
    """读取文件内容"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"读取文件时出错: {e}"

def write_file(input_str: str) -> str:
    """向文件写入内容,输入格式: 'filename.txt,要写入的内容'"""
    try:
        parts = input_str.split(',', 1)
        if len(parts) != 2:
            return "输入格式错误,应为: 'filename.txt,要写入的内容'"
        filename, content = parts
        with open(filename.strip(), 'w', encoding='utf-8') as f:
            f.write(content.strip())
        return f"文件 '{filename.strip()}' 已成功写入。"
    except Exception as e:
        return f"写入文件时出错: {e}"

read_file_tool = Tool(
    name="读取文件",
    func=read_file,
    description="当你需要从文件中读取内容时使用。输入应为文件名。"
)

write_file_tool = Tool(
    name="写入文件",
    func=write_file,
    description="当你需要向文件写入内容时使用。输入应为文件名和内容的组合,格式为 'filename.txt,要写入的内容'。"
)

tools = [search_tool, python_repl_tool, read_file_tool, write_file_tool]
print(f"已加载 {len(tools)} 个工具。")


已加载 4 个工具。


## 5. 记忆：设置向量数据库

为了让智能体拥有长期记忆，我们使用向量数据库。对话历史或重要信息被转换成向量（嵌入），并存储起来。当需要时，智能体可以根据查询的相似性来检索相关记忆。

-   **嵌入模型**: 我们使用 OpenAI 的嵌入模型将文本转换为向量。
-   **向量存储**: 我们使用 `FAISS`，一个高效的相似性搜索库，作为我们的向量数据库。

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

# 1. 初始化嵌入模型
# 使用 ChatAnywhere 的转发服务，嵌入模型也需要设置 base_url
embeddings_model = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    base_url="https://api.chatanywhere.tech/v1"  # 确保嵌入模型也使用转发服务
)

# 2. 初始化 FAISS 向量数据库
# 新版本要求:
# - 第一个参数应该是 Embeddings 对象(不是函数)
# - 使用 embedding_function 参数而不是位置参数
embedding_size = 1536  # OpenAI 嵌入的模型维度
index = faiss.IndexFlatL2(embedding_size)

# 正确的初始化方式
vectorstore = FAISS(
    embedding_function=embeddings_model,  # 传入 Embeddings 对象
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

print("向量数据库记忆模块已初始化。")

向量数据库记忆模块已初始化。



## 6. 双智能体协作系统架构

基于前面构建的基础组件（大脑、工具、记忆），我们现在实现一个双智能体协作系统，专门用于学术论文表达优化。该系统包含：

### 🤖 **Agent A (学术表达优化者)**
- **职责**: 接收用户文本，进行学术化改写和表达优化
- **能力**: 学术化表达、逻辑清晰度提升、语言正式性增强
- **输出**: 优化版本 + 修改策略说明

### 🎓 **Agent B (学术批评与改进建议者)**  
- **职责**: 严格审视Agent A的修改，提出改进建议
- **能力**: 概念清晰度评估、学术规范性检查、逻辑严谨性审查
- **输出**: 具体改进建议 + 优先级排序

### 🔄 **协作流程**
1. 用户提供原文 + 修改要求
2. 5轮迭代：A优化→B审视→A再优化→B再审视...
3. 生成最终版本 + 完整训练数据

In [ ]:
# 🤖 双智能体协作系统核心实现
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableLambda
from typing import Dict, List, Tuple, Optional
import json
import time
from datetime import datetime

class DualAgentAcademicSystem:
    """双智能体学术论文协作系统"""
    
    def __init__(self, llm, tools, vectorstore):
        self.llm = llm  # 主LLM模型
        self.tools = tools  # 工具集合
        self.vectorstore = vectorstore  # 向量存储记忆
        self.chat_history = ChatMessageHistory()
        self.collaboration_log = []  # 协作记录
        
        # 初始化Agent A和B
        self._setup_agents()
        
    def _setup_agents(self):
        """设置双智能体"""
        # Agent A: 学术表达优化者的提示模板
        self.agent_a_template = PromptTemplate.from_template(
            """你是Agent A - 学术表达优化专家。
            
【核心使命】
将普通文本转化为高质量的学术论文表达

【优化维度】
1. 学术化表达：使用准确的学术术语和表达方式
2. 逻辑清晰度：确保论证结构清晰、连贯
3. 语言正式性：符合学术写作的正式性要求
4. 表达精准性：消除歧义，提高表达精确度

【当前任务】
轮次：第{round_num}轮
语言：{language}
用户需求：{user_requirements}

待优化文本：
{text_to_optimize}

{previous_feedback}

请提供优化版本并说明修改理由：
格式：
**优化版本：**
[完整的优化文本]

**修改说明：**
[详细说明本轮的主要修改点和理由]"""
        )
        
        # Agent B: 学术批评者的提示模板
        self.agent_b_template = PromptTemplate.from_template(
            """你是Agent B - 学术批评与改进建议专家。
            
【评审使命】
严格按照学术写作标准，提供具体、可执行的改进建议

【评审维度】
1. 概念清晰度：术语使用、定义明确性
2. 学术规范性：表达是否符合学术标准
3. 逻辑严谨性：论证结构、逻辑链条
4. 语言精确性：用词精准、句式简洁
5. 可读性：表达清晰、层次分明

【当前评审】
轮次：第{round_num}轮
用户原始需求：{user_requirements}

{improvement_history}

Agent A的优化版本：
{optimized_text}

【评审重点】
请重点关注:
1. 与前轮相比的具体改进情况
2. 仍存在的主要问题
3. 下一轮应重点改进的方向
4. 避免重复已经改进的建议

请提供具体改进建议：
格式：
**本轮改进评价：**
[评价本轮Agent A的具体改进，指出哪些建议被采纳，效果如何]

**剩余主要问题：**
[指出仍然存在的最重要的1-2个问题]

**下轮重点建议：**
1. [针对剩余问题的具体、可操作的建议]
2. [新发现的问题和改进建议]

**改进优先级：**
[按紧迫性排序，说明为什么这样排序]"""
        )
        
        # 创建处理链
        self.agent_a_chain = self.agent_a_template | self.llm | StrOutputParser()
        self.agent_b_chain = self.agent_b_template | self.llm | StrOutputParser()
    
    def extract_optimized_text(self, agent_a_response: str) -> str:
        """从Agent A响应中提取优化文本"""
        lines = agent_a_response.split('\n')
        in_optimized_section = False
        optimized_text = []
        
        for line in lines:
            if '**优化版本：**' in line or '优化版本：' in line:
                in_optimized_section = True
                continue
            elif '**修改说明：**' in line or '修改说明：' in line:
                break
            elif in_optimized_section:
                optimized_text.append(line)
        
        return '\n'.join(optimized_text).strip()
    
    def _build_improvement_history(self, current_round):
        """构建改进历史信息"""
        if current_round <= 1:
            return "【首轮评审】这是第一轮评审，请全面评估文本质量。"
        
        history = "【前轮改进历史】\n"
        
        # 获取前面轮次的关键信息
        for i in range(1, current_round):
            if i < len(self.collaboration_log):
                log = self.collaboration_log[i]
                if 'agent_b_feedback' in log:
                    # 提取前轮建议的关键点
                    feedback = log['agent_b_feedback']
                    history += f"第{i}轮建议要点: "
                    
                    # 简化提取建议要点
                    if "改进建议" in feedback:
                        suggestions_part = feedback.split("改进建议")[1].split("优先级排序")[0]
                        # 只保留前100字符作为要点
                        history += suggestions_part[:100].strip() + "...\n"
                    else:
                        history += feedback[:100].strip() + "...\n"
        
        history += "\n请避免重复上述已提出的建议，专注于新问题或未解决的问题。"
        return history
    
    def collaborate(self, user_text: str, user_requirements: str, 
                   language: str = "中文", rounds: int = 5) -> Tuple[str, List[Dict]]:
        """执行双智能体协作优化"""
        print(f"🤖 启动双智能体学术协作系统")
        print(f"📝 原文长度: {len(user_text)} 字符")
        print(f"🎯 用户需求: {user_requirements}")
        print(f"🔄 协作轮次: {rounds}")
        print()
        
        # 初始化协作记录
        self.collaboration_log = [{
            "round": 0,
            "user_input": user_text,
            "user_requirements": user_requirements,
            "timestamp": datetime.now().isoformat()
        }]
        
        current_text = user_text
        previous_feedback = ""
        
        # 执行多轮协作
        for round_num in range(1, rounds + 1):
            print(f"🔄 第 {round_num} 轮协作")
            try:
                # Agent A 优化
                print("  👨‍💼 Agent A 优化中...")
                agent_a_input = {
                    "round_num": round_num,
                    "text_to_optimize": current_text,
                    "user_requirements": user_requirements,
                    "language": language,
                    "previous_feedback": f"前轮建议：\n{previous_feedback}" if previous_feedback else ""
                }
                
                agent_a_response = self.agent_a_chain.invoke(agent_a_input)
                optimized_text = self.extract_optimized_text(agent_a_response)
                
                # Agent B 审视
                print("  👩‍🏫 Agent B 审视中...")
                
                # 构建改进历史信息
                improvement_history = self._build_improvement_history(round_num)
                
                agent_b_input = {
                    "round_num": round_num,
                    "optimized_text": optimized_text,
                    "user_requirements": user_requirements,
                    "improvement_history": improvement_history
                }
                
                agent_b_response = self.agent_b_chain.invoke(agent_b_input)
                
                # 记录本轮协作
                round_log = {
                    "round": round_num,
                    "agent_a_response": agent_a_response,
                    "optimized_text": optimized_text,
                    "agent_b_feedback": agent_b_response,
                    "timestamp": datetime.now().isoformat()
                }
                self.collaboration_log.append(round_log)
                
                # 立即输出本轮迭代日志
                self.display_round_log(round_num, optimized_text, agent_b_response)
                
                # 更新状态
                current_text = optimized_text
                previous_feedback = agent_b_response
                
                print(f"  ✅ 第 {round_num} 轮完成")
                
                # 避免API频率限制
                if round_num < rounds:
                    time.sleep(1)
                    
            except Exception as e:
                print(f"  ❌ 第 {round_num} 轮出错: {e}")
                break
        
        print(f"🎉 协作完成！最终版本长度: {len(current_text)} 字符")
        return current_text, self.collaboration_log
    
    def display_round_log(self, round_num, optimized_text, agent_b_feedback):
        """在每轮协作后立即显示迭代日志"""
        print("\n" + "="*80)
        print(f"📊 Round {round_num} 迭代日志")
        print("="*80)
        
        # 显示本轮变化（如果不是第一轮）
        if round_num > 1 and len(self.collaboration_log) > round_num - 1:
            prev_text = self.collaboration_log[round_num - 1].get('optimized_text', '')
            if prev_text:
                print(f"📈 本轮主要变化：")
                # 简单的差异提示
                if len(optimized_text) != len(prev_text):
                    print(f"   文本长度: {len(prev_text)} → {len(optimized_text)} 字符")
                if optimized_text != prev_text:
                    print(f"   内容已根据建议进行调整")
                else:
                    print(f"   ⚠️  内容未发生变化，可能需要更明确的指导")
                print()
        
        print(f"🤖 Agent A version v{round_num} (完整版本):")
        print(f"{optimized_text}")
        print(f"\n👩‍🏫 Agent B suggestions (完整建议):")
        print(f"{agent_b_feedback}")
        print("="*80 + "\n")
    
    def generate_training_data(self) -> List[Dict]:
        """生成训练数据"""
        training_data = []
        for log in self.collaboration_log[1:]:  # 跳过初始记录
            if "optimized_text" in log:
                training_data.append({
                    "round": log["round"],
                    "input_text": self.collaboration_log[log["round"]-1].get("optimized_text", 
                                                                           self.collaboration_log[0]["user_input"]),
                    "output_text": log["optimized_text"],
                    "feedback": log["agent_b_feedback"],
                    "timestamp": log["timestamp"]
                })
        return training_data

# 创建双智能体系统实例
dual_agent_system = DualAgentAcademicSystem(llm, tools, vectorstore)

print("🤖 双智能体学术协作系统已就绪")
print("✅ Agent A (学术优化者) 和 Agent B (学术批评者) 初始化完成")

IndentationError: unindent does not match any outer indentation level (<string>, line 257)

## 7. 双智能体知识蒸馏：学术写作专业知识快速传递

在双智能体学术协作系统中，我们采用知识蒸馏技术让Agent A（学术优化者）和Agent B（学术批评者）快速获取专业的学术写作知识。通过专家级模型向智能体传递知识，提升协作质量。

**双智能体知识蒸馏流程**:
1.  **Agent A学习**: 让Agent A先尝试优化学术文本，获得初步结果
2.  **专家评估**: 专家级模型评估Agent A的优化效果，提供专业指导
3.  **Agent B验证**: Agent B根据专家指导重新审视优化结果
4.  **知识固化**: 将专家知识集成到双智能体的协作流程中

这种方法能够让我们的双智能体系统快速掌握学术写作的专业知识和最佳实践。

In [7]:
# 🎓 双智能体知识蒸馏：学术写作专业知识传递系统
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

class AcademicKnowledgeDistillation:
    """双智能体学术写作知识蒸馏系统"""
    
    def __init__(self, dual_agent_system):
        self.dual_agent_system = dual_agent_system
        
        # 专家级学术写作模型 (知识源)
        self.expert_model = ChatOpenAI(
            temperature=0.1,  # 低温度确保稳定输出
            model_name="gpt-4o-mini",
            openai_api_key=openai_api_key,
            base_url="https://api.chatanywhere.tech/v1",
            max_tokens=1500
        )
        
        # Agent A的学习模型 (知识接受者)
        self.agent_a_learner = ChatOpenAI(
            temperature=0.3,
            model_name="gpt-3.5-turbo", 
            openai_api_key=openai_api_key,
            base_url="https://api.chatanywhere.tech/v1",
            max_tokens=1000
        )
    
    def create_expert_prompt(self):
        """创建专家评估提示模板"""
        return PromptTemplate.from_template(
            """你是顶级的学术写作专家，拥有丰富的期刊发表和同行评议经验。
            
【任务】评估Agent A的学术文本优化结果，并提供专业指导

【评估维度】
1. 学术规范性：是否符合学术写作标准
2. 逻辑清晰度：论证结构是否合理
3. 表达精确性：术语使用是否准确
4. 可读性：是否易于理解

【原始文本】
{original_text}

【Agent A优化版本】 
{agent_a_version}

【用户需求】
{user_requirements}

请提供专业评估和改进指导：

**专业评分（1-10分）：**
学术规范性：[分数]/10
逻辑清晰度：[分数]/10 
表达精确性：[分数]/10
可读性：[分数]/10

**专家改进版本：**
[提供您作为专家的优化版本]

**关键知识点：**
[总结Agent A需要掌握的核心学术写作知识]

**具体指导建议：**
[为Agent A提供具体、可操作的改进建议]"""
        )
    
    def create_agent_learning_prompt(self):
        """创建Agent学习提示模板"""
        return PromptTemplate.from_template(
            """你是Agent A，正在学习专业的学术写作技能。专家已经对你的优化结果进行了评估。

【学习任务】根据专家指导，重新优化文本并内化专业知识

【专家评估】
{expert_feedback}

【原始文本】
{original_text}

【你的前一版本】
{previous_version}

请根据专家指导进行学习和改进：

**知识吸收总结：**
[总结从专家指导中学到的关键知识点]

**改进版本：**
[基于专家指导的新优化版本]

**学习反思：**
[反思此次学习的收获和今后注意事项]"""
        )
    
    def distill_knowledge(self, original_text, agent_a_version, user_requirements):
        """执行知识蒸馏过程"""
        print("🎓 启动双智能体知识蒸馏流程")
        print("=" * 70)
        print(f"📝 原文: {original_text[:100]}...")
        print(f"🤖 Agent A版本: {agent_a_version[:100]}...")
        print()
        
        # 创建处理链
        expert_prompt = self.create_expert_prompt()
        learning_prompt = self.create_agent_learning_prompt()
        
        expert_chain = expert_prompt | self.expert_model | StrOutputParser()
        learning_chain = learning_prompt | self.agent_a_learner | StrOutputParser()
        
        try:
            # 第一步：专家评估Agent A的优化结果
            print("👨‍🏫 专家正在评估Agent A的优化结果...")
            
            expert_feedback = expert_chain.invoke({
                "original_text": original_text,
                "agent_a_version": agent_a_version,
                "user_requirements": user_requirements
            })
            
            print("✅ 专家评估完成")
            
            # 第二步：Agent A根据专家指导学习改进
            print("🤖 Agent A正在根据专家指导学习改进...")
            
            learning_result = learning_chain.invoke({
                "expert_feedback": expert_feedback,
                "original_text": original_text,
                "previous_version": agent_a_version
            })
            
            print("✅ Agent A学习完成")
            
            # 第三步：让Agent B验证学习效果
            print("🎯 Agent B正在验证学习效果...")
            
            # 从学习结果中提取改进版本
            improved_version = self.extract_improved_version(learning_result)
            
            # 使用双智能体系统的Agent B进行验证
            agent_b_input = {
                "round_num": "知识蒸馏验证",
                "optimized_text": improved_version,
                "user_requirements": user_requirements
            }
            
            agent_b_verification = self.dual_agent_system.agent_b_chain.invoke(agent_b_input)
            
            print("✅ Agent B验证完成")
            
            return {
                "expert_feedback": expert_feedback,
                "learning_result": learning_result,
                "improved_version": improved_version,
                "agent_b_verification": agent_b_verification
            }
            
        except Exception as e:
            print(f"❌ 知识蒸馏失败: {e}")
            return None
    
    def extract_improved_version(self, learning_result):
        """从学习结果中提取改进版本"""
        lines = learning_result.split('\n')
        in_improved_section = False
        improved_text = []
        
        for line in lines:
            if '**改进版本：**' in line or '改进版本：' in line:
                in_improved_section = True
                continue
            elif '**学习反思：**' in line or '学习反思：' in line:
                break
            elif in_improved_section:
                improved_text.append(line)
        
        return '\n'.join(improved_text).strip()
    
    def display_distillation_report(self, distillation_result):
        """显示知识蒸馏报告"""
        if not distillation_result:
            print("❌ 无蒸馏结果可显示")
            return
        
        print("\n📊 知识蒸馏报告")
        print("=" * 70)
        
        print("\n👨‍🏫 专家评估与指导：")
        print("-" * 40)
        print(distillation_result["expert_feedback"])
        
        print("\n🤖 Agent A学习成果：")
        print("-" * 40) 
        print(distillation_result["learning_result"])
        
        print("\n🎯 Agent B验证结果：")
        print("-" * 40)
        print(distillation_result["agent_b_verification"])
        
        print("\n✨ 最终改进版本：")
        print("-" * 40)
        print(distillation_result["improved_version"])

# 创建知识蒸馏系统
print("🚀 初始化双智能体知识蒸馏系统...")

if 'dual_agent_system' in globals():
    knowledge_distillation = AcademicKnowledgeDistillation(dual_agent_system)
    print("✅ 知识蒸馏系统初始化完成")

else:
    print("❌ 请先运行双智能体系统初始化")

🚀 初始化双智能体知识蒸馏系统...
✅ 知识蒸馏系统初始化完成


In [8]:
# 🧠 知识持久化与Agent动态增强系统 
from langchain_core.documents import Document

class PersistentKnowledgeSystem:
    """知识持久化与Agent动态增强系统"""
    
    def __init__(self, dual_agent_system, vectorstore, knowledge_distillation):
        self.dual_agent_system = dual_agent_system
        self.vectorstore = vectorstore
        self.knowledge_distillation = knowledge_distillation
        self.knowledge_base = []  # 存储结构化知识
        self.agent_enhancements = {
            "agent_a_knowledge": [],
            "agent_b_knowledge": [],
            "expert_insights": []
        }
    
    def store_distillation_knowledge(self, distillation_result, task_type="学术写作"):
        """存储蒸馏过程中获得的知识"""
        if not distillation_result:
            return False
        
        print("💾 存储蒸馏知识到持久化系统...")
        
        # 提取关键知识点
        expert_feedback = distillation_result["expert_feedback"]
        learning_result = distillation_result["learning_result"]
        
        # 构建知识条目
        knowledge_entry = {
            "timestamp": datetime.now().isoformat(),
            "task_type": task_type,
            "expert_feedback": expert_feedback,
            "learning_result": learning_result,
            "improved_version": distillation_result["improved_version"],
            "knowledge_points": self.extract_knowledge_points(expert_feedback, learning_result)
        }
        
        # 存储到知识库
        self.knowledge_base.append(knowledge_entry)
        
        # 存储到向量数据库（用于相似性检索）
        knowledge_text = f"任务类型: {task_type}\\n专家反馈: {expert_feedback}\\n学习成果: {learning_result}"
        knowledge_doc = Document(
            page_content=knowledge_text,
            metadata={
                "type": "distillation_knowledge",
                "task_type": task_type,
                "timestamp": knowledge_entry["timestamp"]
            }
        )
        
        self.vectorstore.add_documents([knowledge_doc])
        print("✅ 知识存储完成")
        return True
    
    def extract_knowledge_points(self, expert_feedback, learning_result):
        """从专家反馈和学习结果中提取关键知识点"""
        knowledge_points = []
        
        # 从专家反馈中提取关键知识点
        if "关键知识点" in expert_feedback:
            expert_section = expert_feedback.split("关键知识点")[1].split("具体指导建议")[0]
            knowledge_points.append(f"专家知识: {expert_section.strip()}")
        
        # 从学习结果中提取
        if "知识吸收总结" in learning_result:
            learning_section = learning_result.split("知识吸收总结")[1].split("改进版本")[0]
            knowledge_points.append(f"学习总结: {learning_section.strip()}")
        
        return knowledge_points
    
    def get_relevant_knowledge(self, user_text, user_requirements, top_k=2):
        """获取与当前任务相关的历史知识"""
        
        query_text = f"用户文本: {user_text}\\n用户需求: {user_requirements}"
        
        try:
            # 从向量数据库检索相关知识
            relevant_docs = self.vectorstore.similarity_search(
                query_text, 
                k=top_k
            )
            
            relevant_knowledge = []
            for doc in relevant_docs:
                if doc.metadata.get("type") == "distillation_knowledge":
                    relevant_knowledge.append({
                        "content": doc.page_content,
                        "metadata": doc.metadata
                    })
            
            return relevant_knowledge
        except Exception as e:
            print(f"⚠️ 知识检索失败: {e}")
            return []
    
    def enhanced_collaborate(self, user_text, user_requirements, language="中文", rounds=3):
        """使用知识增强的协作系统"""
        
        print("🧠 启动知识增强型双智能体协作")
        print("=" * 70)
        
        # 获取相关历史知识
        relevant_knowledge = self.get_relevant_knowledge(user_text, user_requirements)
        
        if relevant_knowledge:
            print(f"📚 检索到 {len(relevant_knowledge)} 条相关历史知识")
            
            # 构建知识增强提示
            knowledge_text = "\\n【历史专家经验】\\n"
            for i, knowledge in enumerate(relevant_knowledge, 1):
                knowledge_text += f"{i}. {knowledge['content'][:150]}...\\n"
            
            # 临时增强Agent A的提示模板
            original_template = self.dual_agent_system.agent_a_template.template
            enhanced_template = original_template + knowledge_text + "\\n请结合以上专家经验进行优化。"
            
            # 创建增强型Agent A
            enhanced_agent_a_template = PromptTemplate.from_template(enhanced_template)
            enhanced_agent_a_chain = enhanced_agent_a_template | self.dual_agent_system.llm | StrOutputParser()
            
            # 临时替换Agent A的链
            original_agent_a_chain = self.dual_agent_system.agent_a_chain
            self.dual_agent_system.agent_a_chain = enhanced_agent_a_chain
            
            try:
                # 执行增强协作
                final_text, collaboration_log = self.dual_agent_system.collaborate(
                    user_text, user_requirements, language, rounds
                )
                
                print("✅ 知识增强协作完成")
                return final_text, collaboration_log
                
            finally:
                # 恢复原有Agent A链
                self.dual_agent_system.agent_a_chain = original_agent_a_chain
        else:
            print("📝 未找到相关历史知识，使用标准协作模式")
            return self.dual_agent_system.collaborate(user_text, user_requirements, language, rounds)
    
    def display_knowledge_stats(self):
        """显示知识库统计信息"""
        print("📊 持久化知识库统计")
        print("=" * 50)
        print(f"💾 总知识条目: {len(self.knowledge_base)}")
        
        if self.knowledge_base:
            print("\\n📋 最近的知识条目:")
            latest = self.knowledge_base[-1]
            print(f"   时间: {latest['timestamp']}")
            print(f"   类型: {latest['task_type']}")
            print(f"   知识点数: {len(latest['knowledge_points'])}")

# 创建知识持久化系统
if all(var in globals() for var in ['dual_agent_system', 'vectorstore', 'knowledge_distillation']):
    persistent_knowledge = PersistentKnowledgeSystem(dual_agent_system, vectorstore, knowledge_distillation)

## 📊 用户交互与结果查看模块

本模块提供了用户友好的交互界面，允许用户输入需求并查看详细的处理结果。包含：

1. **最终版本输出** - 适合论文发表的最终版本
2. **详细日志输出** - 包含完整迭代过程，可用于微调训练数据

### 功能特点
- 🎯 交互式用户输入界面
- 📄 格式化的最终输出
- 📝 详细的迭代日志
- 💾 可导出的训练数据格式

In [9]:
class InteractiveResultViewer:
    """
    交互式结果查看器
    提供用户友好的输入界面和格式化的输出显示
    """
    
    def __init__(self, persistent_knowledge_system):
        self.pks = persistent_knowledge_system
        self.session_history = []
    
    def get_user_input(self):
        """获取用户输入"""
        print("="*60)
        print("🎯 多Agent学术写作优化系统")
        print("="*60)
        print("请输入您的需求：")
        print("💡 提示：您可以输入学术文本、研究想法或写作要求")
        print("-"*60)
        
        user_text = input("📝 请输入文本内容: ").strip()
        if not user_text:
            return None, None, None, None
        
        print("\n请选择处理要求（可多选，用逗号分隔）：")
        print("1️⃣  语法检查与修正")
        print("2️⃣  逻辑结构优化") 
        print("3️⃣  学术表达提升")
        print("4️⃣  参考文献格式化")
        print("5️⃣  创新性增强")
        print("6️⃣  可读性改进")
        print("7️⃣  自定义要求")
        
        requirements_input = input("📋 选择要求 (例: 1,3,5 或直接输入自定义要求): ").strip()
        
        # 处理要求选择
        requirements_map = {
            "1": "语法检查与修正",
            "2": "逻辑结构优化", 
            "3": "学术表达提升",
            "4": "参考文献格式化",
            "5": "创新性增强",
            "6": "可读性改进"
        }
        
        if requirements_input.replace(",", "").replace(" ", "").isdigit():
            selected_nums = [num.strip() for num in requirements_input.split(",")]
            user_requirements = [requirements_map.get(num, "") for num in selected_nums if num in requirements_map]
            user_requirements = [req for req in user_requirements if req]
        else:
            user_requirements = [requirements_input]
        
        if not user_requirements:
            user_requirements = ["学术表达提升"]
        
        # 选择语言
        print("\n请选择输出语言：")
        print("🌍 1. 中文")
        print("🌎 2. English")
        language_choice = input("🗣️  语言选择 (1/2): ").strip()
        language = "中文" if language_choice != "2" else "English"
        
        # 选择迭代轮数
        print("\n请选择优化轮数：")
        print("⚡ 1. 快速模式 (3轮)")
        print("🎯 2. 标准模式 (5轮)")  
        print("🔄 3. 深度模式 (8轮)")
        rounds_choice = input("🔄 选择模式 (1/2/3): ").strip()
        
        rounds_map = {"1": 3, "2": 5, "3": 8}
        rounds = rounds_map.get(rounds_choice, 5)
        
        return user_text, user_requirements, language, rounds
    
    def format_final_output(self, final_text, collaboration_log):
        """格式化最终输出结果"""
        print("\n" + "="*80)
        print("📋 最终优化结果 (适合论文发表)")
        print("="*80)
        print(final_text)
        print("="*80)
        
        return final_text
    
    def format_training_data(self, collaboration_log, user_input_info):
        """构建训练数据（不再输出，因为已在每轮协作时输出）"""
        training_data = []
        
        for i, round_info in enumerate(collaboration_log, 1):
            # 跳过初始用户输入记录
            if 'optimized_text' not in round_info:
                continue
                
            # 构建训练数据条目
            training_entry = {
                "round": i,
                "agent_a_version": round_info.get('optimized_text', ''),
                "agent_b_suggestions": round_info.get('agent_b_feedback', ''),
                "timestamp": round_info.get('timestamp', ''),
            }
            training_data.append(training_entry)
        
        return training_data
    
    def run_interactive_session(self):
        """运行交互式会话"""
        from datetime import datetime
        
        # 获取用户输入
        user_input_info = self.get_user_input()
        if user_input_info[0] is None:
            print("❌ 输入为空，会话结束")
            return
        
        user_text, user_requirements, language, rounds = user_input_info
        
        print(f"\n🚀 开始处理...")
        print(f"📝 文本长度: {len(user_text)} 字符")
        print(f"🎯 要求: {', '.join(user_requirements)}")
        print(f"🌍 语言: {language}")
        print(f"🔄 轮数: {rounds}")
        print("-"*60)
        
        try:
            # 使用持久化知识系统处理
            final_text, collaboration_log = self.pks.enhanced_collaborate(
                user_text, user_requirements, language, rounds
            )
            
            # 格式化并显示最终结果
            formatted_final = self.format_final_output(final_text, collaboration_log)
            
            # 构建训练数据（用于历史记录）
            training_data = self.format_training_data(collaboration_log, user_input_info)
            
            # 保存到会话历史
            session_record = {
                "timestamp": datetime.now().isoformat(),
                "user_input": user_input_info,
                "final_result": final_text,
                "training_data": training_data,
                "exported_file": None
            }
            self.session_history.append(session_record)
            
            print(f"\n✅ 会话完成! 总计处理轮数: {rounds}")
            
        except Exception as e:
            print(f"❌ 处理过程中出现错误: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def show_session_history(self):
        """显示会话历史"""
        print("\n📋 会话历史")
        print("="*60)
        
        if not self.session_history:
            print("暂无历史记录")
            return
        
        for i, session in enumerate(self.session_history, 1):
            print(f"{i}. {session['timestamp']}")
            user_text = session['user_input'][0]
            print(f"   输入: {user_text[:50]}..." if len(user_text) > 50 else f"   输入: {user_text}")
            print()

# 创建交互式结果查看器
if 'persistent_knowledge' in globals():
    interactive_viewer = InteractiveResultViewer(persistent_knowledge)
    print("✅ 交互式结果查看器已创建")
else:
    print("❌ 请先运行持久化知识系统初始化")

✅ 交互式结果查看器已创建


### 🎯 使用示例

以下是如何使用交互式结果查看器的示例：

In [10]:
# 🎯 运行交互式会话
# 这将启动用户友好的交互界面
if 'interactive_viewer' in globals():
    print("🚀 启动交互式多Agent学术写作优化系统")
    print("💡 使用方法: interactive_viewer.run_interactive_session()")
    print("📋 查看历史: interactive_viewer.show_session_history()")
    print("📊 知识库状态: persistent_knowledge.display_knowledge_stats()")
    print()
    print("=" * 60)
    print("🔥 系统就绪! 请调用以下命令开始使用:")
    print("   interactive_viewer.run_interactive_session()")
    print("=" * 60)
else:
    print("❌ 请确保已执行前面的所有代码单元格")

🚀 启动交互式多Agent学术写作优化系统
💡 使用方法: interactive_viewer.run_interactive_session()
📋 查看历史: interactive_viewer.show_session_history()
📊 知识库状态: persistent_knowledge.display_knowledge_stats()

🔥 系统就绪! 请调用以下命令开始使用:
   interactive_viewer.run_interactive_session()


### 🚀 快速启动

执行以下命令开始使用系统：

In [11]:
# 🚀 开始交互式会话
# 取消下面这行的注释来启动交互式界面
interactive_viewer.run_interactive_session()

# 📊 查看知识库统计信息  
#persistent_knowledge.display_knowledge_stats()

# 📋 查看会话历史
#interactive_viewer.show_session_history()

print("🎯 多Agent学术写作优化系统已就绪!")
print("💡 取消上面任意一行的注释来开始使用相应功能")
print("📊 系统组件状态:")
print(f"   ✅ 双Agent系统: {'已加载' if 'dual_agent_system' in globals() else '未加载'}")
print(f"   ✅ 知识持久化: {'已加载' if 'persistent_knowledge' in globals() else '未加载'}")
print(f"   ✅ 交互式查看器: {'已加载' if 'interactive_viewer' in globals() else '未加载'}")
print(f"   ✅ 向量存储: {'已加载' if 'vectorstore' in globals() else '未加载'}")
print(f"   ✅ 知识蒸馏: {'已加载' if 'knowledge_distillation' in globals() else '未加载'}")

🎯 多Agent学术写作优化系统
请输入您的需求：
💡 提示：您可以输入学术文本、研究想法或写作要求
------------------------------------------------------------

请选择处理要求（可多选，用逗号分隔）：
1️⃣  语法检查与修正
2️⃣  逻辑结构优化
3️⃣  学术表达提升
4️⃣  参考文献格式化
5️⃣  创新性增强
6️⃣  可读性改进
7️⃣  自定义要求

请选择处理要求（可多选，用逗号分隔）：
1️⃣  语法检查与修正
2️⃣  逻辑结构优化
3️⃣  学术表达提升
4️⃣  参考文献格式化
5️⃣  创新性增强
6️⃣  可读性改进
7️⃣  自定义要求

请选择输出语言：
🌍 1. 中文
🌎 2. English

请选择输出语言：
🌍 1. 中文
🌎 2. English

请选择优化轮数：
⚡ 1. 快速模式 (3轮)
🎯 2. 标准模式 (5轮)
🔄 3. 深度模式 (8轮)

请选择优化轮数：
⚡ 1. 快速模式 (3轮)
🎯 2. 标准模式 (5轮)
🔄 3. 深度模式 (8轮)

🚀 开始处理...
📝 文本长度: 176 字符
🎯 要求: 逻辑结构优化, 学术表达提升, 创新性增强, 可读性改进
🌍 语言: 中文
🔄 轮数: 3
------------------------------------------------------------
🧠 启动知识增强型双智能体协作

🚀 开始处理...
📝 文本长度: 176 字符
🎯 要求: 逻辑结构优化, 学术表达提升, 创新性增强, 可读性改进
🌍 语言: 中文
🔄 轮数: 3
------------------------------------------------------------
🧠 启动知识增强型双智能体协作
📝 未找到相关历史知识，使用标准协作模式
🤖 启动双智能体学术协作系统
📝 原文长度: 176 字符
🎯 用户需求: ['逻辑结构优化', '学术表达提升', '创新性增强', '可读性改进']
🔄 协作轮次: 3

🔄 第 1 轮协作
  👨‍💼 Agent A 优化中...
📝 未找到相关历史知识，使用标准协作模式
🤖 启动双智能体学术协作系统
📝 原文长度: 176 字符
🎯 用户需求: